<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/dev/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-whisper

This Project uses the Original [Whisper](https://github.com/openai/whisper) Project instead of Decipher Project.

For Everyone who want to use the Original Whisper Project

# Usage

1. Check the Runtime Type is on GPU Mode in ``Runtime -> Change Runtime Type ``
1. Change the Settings at Settings Code Block
1. Click ``Runtime -> Run all`` (CTRL+F9)
1. Click on the Connect to Google Drive
1. And Wait for a moment your Generated Srt should be on ``Google Drive -> Whisper -> result``

## Add Google Drive Access 
(at Google Drive/Whisper)

In [1]:
from google.colab import drive
import os

path = '/content/gdrive/MyDrive/Whisper/'
path2 = '/content/gdrive/MyDrive/Whisper/result'
drive.mount('/content/gdrive')

if not os.path.exists(path):
  os.mkdir(path)
  os.mkdir(path2)


Mounted at /content/gdrive


# Install Components

Package : yt-dlp ffmpeg openai-whisper

In [2]:
! pip install openai-whisper yt-dlp ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 44.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 108.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 101.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=3be0962da708c741125b275829d780db827475b252bdd4e31828be744b

# GPU Checkup

If Nothing shows up means you didn't enable GPU in the ``Runtime -> Change Runtime type``

In [3]:
! nvidia-smi

Tue May 30 10:03:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Settings


In [4]:
#@markdown # **Video Link Here**
# YouTube Link
YouTube_Video_Link = "https://www.youtube.com/watch?v=UmXNI9PcxDI" #@param {type:"string"}
# File Name Change
new_filename = "Alpine Linux - Simple, Small, and Secure" #@param {type:"string"}
#@markdown Note: Leave it Blank to let it audio Detect the Language
language = "english" #@param {type:"string"}
task = "transcribe" #@param ['transcribe', 'translate']
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
new_filename2 = os.path.splitext(new_filename)[0] + " Transcript.txt"
# Change the Model of the Whisper
#@markdown # **Other Options**
Generate_Plain_Document = "yes" #@param ['yes','no']
shutdown_after_complete = "yes" #@param ['yes','no']
model_user = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
enable_cookies = "no" #@param ['yes', 'no']

# If Enable Cookies is Enabled
Upload your Cookies file in here

In [5]:
import os
if enable_cookies.lower() == "yes":
  print("It is not recommended to upload your main account's cookies files here, as this is a shared environment.")
  print("Your cookies might get leaked, and I will not take any responsibility if your account gets stolen.")
  print("The recommended approach is to use another Google Account's cookies to run this tool.")
  print("If it is a private video, you can share the video with that Google Account to grant access to it.")
  print("")

  from google.colab import files
  cookies = files.upload()

  # Specify the path where the files are located
  path_cookies = "/content"

  def rename_files(path_cookies):
      for filename in os.listdir(path_cookies):
          file_path = os.path.join(path_cookies, filename)
          if os.path.isfile(file_path) and filename.endswith(".txt"):
              new_path = os.path.join(path_cookies, "cookies.txt")
              os.rename(file_path, new_path)

  rename_files(path_cookies)

# Download the Video
using the yt-dlp and ffmpeg

In [6]:
import yt_dlp

if enable_cookies.lower() == "yes":

  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
          'cookiefile': 'cookies.txt'
  }

else:

    ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
          'outtmpl': 'content/audio',
  }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=UmXNI9PcxDI
[youtube] UmXNI9PcxDI: Downloading webpage
[youtube] UmXNI9PcxDI: Downloading android player API JSON
[info] UmXNI9PcxDI: Downloading 1 format(s): 251
[dashsegments] Total fragments: 2
[download] Destination: content/audio
[download] 100% of   10.31MiB in 00:00:00 at 53.77MiB/s              
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# Generating the SRT file

In [ ]:
import whisper

model = whisper.load_model(model_user)

print("Whisper model loaded.")


from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"

result = model.transcribe(input_file, verbose=True, task=task, language=language)
transcription_root = "/content/gdrive/MyDrive/Whisper/result"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, new_filename)

if Generate_Plain_Document.lower() == "yes":

  # Save as TXT file
  srt_writer = get_writer("txt", transcription_root,)
  srt_writer(result, new_filename2)
  print("Generated srt and txt file in the Google Drive -> Whisper -> result Folder")

else:

  print("Generated srt file in the Google Drive -> Whisper -> result Folder")

100%|██████████████████████████████████████| 1.42G/1.42G [00:07<00:00, 210MiB/s]


Whisper model loaded.
[00:00.000 --> 00:05.120]  You know what I need to cover? Alpine Linux. I've installed Alpine probably more than any other
[00:05.120 --> 00:09.600]  Linux distribution. I rarely ever use the damn thing. Alpine's are really interesting. Like if
[00:09.600 --> 00:17.600]  you want like a really simple, basic one for like, you know, you're putting it on a potato.
[00:19.360 --> 00:26.960]  Alpine Linux is kind of what I use for that. Small, simple, secure. Dude, now you're speaking my
[00:26.960 --> 00:33.280]  language. Alpine's great. So if you have like a netbook or something like that, you just,
[00:33.280 --> 00:37.440]  you can't find anything better than Alpine. I think it's like a hundred megs. Let's just download
[00:37.440 --> 00:45.520]  it and just see. I mean, it's something I toss when I need just like a base Linux. It has 64-bit,
[00:45.520 --> 00:52.720]  it has ARM, it has PowerPC. You know, you got your traditional right here. Let's just grab the
[

# Auto Shutdown the Runtime

In [ ]:
if shutdown_after_complete.lower() == "yes":
    print("Shutting Down the Runtime Because Shutdown After Complete is on")
    from google.colab import runtime
    runtime.unassign()